# Introduction into web evaluation setup

## Note

It is highly recommended to checkout [setup_and_intro.ipynb](https://github.com/metno/pyaerocom-meetings/blob/master/Feb2021_Workshop/setup_and_intro.ipynb) and make sure all is in place to use pyaerocom with access to PPI.

## Setting up the configuration for the analysis

In [1]:
import os
import pyaerocom as pya
pya.__version__

'0.10.0'

In the following, a whole configuration setup is specified, see comments for details.

In [2]:
# ID of project
PROJ_ID = 'workshop2021'

# ID of experiment 
EXP_ID = 'tutorial'

# Output directory (where json files are stored)
OUT_BASEDIR = os.path.abspath('../../data/json/')

# Directory where colocated NetCDF files are stored
COLDATA_BASEDIR = os.path.abspath('./coldata')

In [3]:
stp = pya.web.AerocomEvaluation(proj_id=PROJ_ID, exp_id=EXP_ID, 
                                exp_name='Example configuration for pyaerocom workshop',
                                out_basedir=OUT_BASEDIR,
                                basedir_coldata=COLDATA_BASEDIR)
print(stp)


Pyaerocom AerocomEvaluation
---------------------------
Project ID: workshop2021
Eperiment ID: tutorial
Experiment name: Example configuration for pyaerocom workshop
colocation_settings: (will be updated for each run from model_config and obs_config entry)
  save_coldata: True
  _obs_cache_only: False
  obs_vars: None
  obs_vert_type: None
  model_vert_type_alt: None
  read_opts_ungridded: None
  obs_ts_type_read: None
  model_use_vars: None
  model_add_vars: None
  model_keep_outliers: True
  model_to_stp: False
  model_id: None
  model_name: None
  model_data_dir: None
  obs_id: None
  obs_name: None
  obs_data_dir: None
  obs_keep_outliers: False
  obs_use_climatology: False
  obs_add_meta: []
  gridded_reader_id: {'model': 'ReadGridded', 'obs': 'ReadGridded'}
  start: None
  stop: None
  ts_type: None
  filter_name: None
  remove_outliers: True
  apply_time_resampling_constraints: None
  min_num_obs: None
  resample_how: None
  var_outlier_ranges: None
  var_ref_outlier_ranges: No

The most important things to define for the analysis are:

- obs_config: dictionary of dictionaries containing observations to be used
- model_config: dictionary of dictionaries containing models to be used
- colocation_settings: (see above) most of these can be left untouched and below we show the essential information

### Observation setup

The `obs_config` entry defines observations to be used, below we define 1 set of observations, Aeronet (AOD and Angstrom Exponent) and EBAS (scattering and absorption coefficients). In the end, these setups are assigned to the evaluation class that we just created.

In [4]:
obs_cfg = {
    # key is name as it appears in web interface, value contains setup 
    'Aeronet' : {
        'obs_id'        : 'AeronetSunV3Lev2.daily', # ID of obsnetwork
        'obs_vars'      : ['ang4487aer', 'od550aer'], # list of variables (Angstrom Exponent, 440-870nm, and AOD at 550 nm)
        'obs_vert_type' : 'Column', # this is needed, choose from Column or Surface
        'obs_filters'   : {'altitude' : [0, 1000]},
        'ignore_station_names' : 'DRAGON*'
    }    
}

stp['obs_config'] = obs_cfg

### Defining models to be used for evaluation

In [5]:
model_cfg = {
    'Aerocom-Median' : {'model_id' : 'AEROCOM-MEDIAN-2x3-GLISSETAL2020-1_AP3-CTRL'},
    'EC-Earth'    : {'model_id' : 'EC-Earth3-AerChem-met2010_AP3-CTRL2019'}
}

stp['model_config'] = model_cfg

## Colocation setup

In the following we define essential settings for colocation of model / obs / var. Note: these can be overwritten in each individual model or obs config entry where needed.

In [6]:
DEFAULT_COLOCATION_SETTINGS = dict(
    start = 2010, 
    stop = 2011,
    ts_type = 'daily', # desired output frequency of colocated data objects
    colocate_time = False,
    weighted_stats = True, # only relevant if models are evaluated against gridded satellite data
    apply_time_resampling_constraints = True,
    min_num_obs = pya.const.OBS_MIN_NUM_RESAMPLE,
    reanalyse_existing = False, # relevant for re-runs. If True, pre-existing colocated data files are re-used for computation of json files 
    remove_outliers=True, # remove outliers during colocation
    harmonise_units=True,
    model_keep_outliers=True,   
)

stp.update(**DEFAULT_COLOCATION_SETTINGS)

In [7]:
print(stp)


Pyaerocom AerocomEvaluation
---------------------------
Project ID: workshop2021
Eperiment ID: tutorial
Experiment name: Example configuration for pyaerocom workshop
colocation_settings: (will be updated for each run from model_config and obs_config entry)
  save_coldata: True
  _obs_cache_only: False
  obs_vars: None
  obs_vert_type: None
  model_vert_type_alt: None
  read_opts_ungridded: None
  obs_ts_type_read: None
  model_use_vars: None
  model_add_vars: None
  model_keep_outliers: True
  model_to_stp: False
  model_id: None
  model_name: None
  model_data_dir: None
  obs_id: None
  obs_name: None
  obs_data_dir: None
  obs_keep_outliers: False
  obs_use_climatology: False
  obs_add_meta: []
  gridded_reader_id: {'model': 'ReadGridded', 'obs': 'ReadGridded'}
  start: 2010
  stop: 2011
  ts_type: daily
  filter_name: None
  remove_outliers: True
  apply_time_resampling_constraints: True
  min_num_obs: {'yearly': {'monthly': 3}, 'monthly': {'daily': 7}, 'daily': {'hourly': 6}, 'hou

In [8]:
stp.var_mapping = pya.web.web_naming_conventions.VAR_MAPPING

In [9]:
stp.run_evaluation()


Running analysis:
Obs. names: ['Aeronet']
Model names: ['Aerocom-Median', 'EC-Earth']
Remove outliers: True
Harmonise units: True
Delete existing json files before reanalysis: True
Reanalyse existing colocated NetCDF files: False
Run only colocation (no json files computed): False
Raise exceptions if they occur: False

Running colocation of Aerocom-Median against Aeronet
PREPARING colocation of AEROCOM-MEDIAN-2x3-GLISSETAL2020-1_AP3-CTRL vs. AeronetSunV3Lev2.daily
The following variable combinations will be colocated
MODEL-VAR	OBS-VAR
ang4487aer	ang4487aer
od550aer	od550aer
Running AEROCOM-MEDIAN-2x3-GLISSETAL2020-1_AP3-CTRL / AeronetSunV3Lev2.daily (ang4487aer, ang4487aer)
Updating ts_type from daily to monthly (highest available in model AEROCOM-MEDIAN-2x3-GLISSETAL2020-1_AP3-CTRL)
Input filters {'altitude': [-1000000.0, 1000.0]} result in unchanged data object
Input filters {'latitude': [-89.0, 89.0], 'longitude': [-178.5, 178.5]} result in unchanged data object
WRITE: ang4487aer_R

['/home/jonasg/MyPyaerocom/ws21/pyaerocom-meetings/Feb2021_Workshop/coldata/EC-Earth3-AerChem-met2010_AP3-CTRL2019/ang4487aer_REF-Aeronet_MOD-EC-Earth_20100101_20101231_monthly_None.nc',
 '/home/jonasg/MyPyaerocom/ws21/pyaerocom-meetings/Feb2021_Workshop/coldata/EC-Earth3-AerChem-met2010_AP3-CTRL2019/od550aer_REF-Aeronet_MOD-EC-Earth_20100101_20101231_monthly_None.nc']